# Fuzzy Matching 

## Introduction

The whole idea of getting a computer to compare two seperate strings can seem like an easy task, after all it is extremely easy for us as humans to identify whether or not a word matches another at just a glance. However, this tasks becomes cumbersome when faced with the challenge to compare thousands of addresses to one another. Not only will a normal human mind lose interest but performance may suffer as well. 

Fuzzy matching isn't anything new. This is a problem that has been solved in a variety of ways (however, you would be suprised at how many people do not know that these tools exists). Due to the fact that fuzzy matching is a asked and answered questions there are packages available that one can pip install and just be a simple "import and call in" away from having your own fuzzy match. The python package Fuzzy Wuzzy is an example of one. However I really wanted to dive into how these algorithms work. I wanted to shed a little light on this "black box". So, I will attempt to code a fuzzy match from scratch. 

Now there are a few techniques for comparing two seperate strings and they mostly involve calculating the distance between the two words. Some of these techniques include: Levenshtein distance, Damerau-Levenshtein distance, Bitap, n-gram, BK-tree, Soundex, and Jaro-winkler distance. Many of these algorithms can be found in open sources libraries and are useful tools. I on the other hand want to code one of these from scratch. I chose to implement Levenshtein disantance for a few reasons. From my research it appeared that many of the other models would not be as efficient as levenshteins implementation, or they served a slightly different purpose than what I was trying to achieve. 

## Outside Sources

Here is a wonderful article that will walk you through the Levenshtein Algorithm: https://www.cuelogic.com/blog/the-levenshtein-algorithm

Here you will find definitions along with the strength and weaknesses of each algorithm: https://www.datasciencecentral.com/profiles/blogs/fuzzy-matching-algorithms-to-help-data-scientists-match-similar

## The Problem

So with this fuzzy match I want to be able to identify how similar two address are to one another. From my research in application fraud ring behavior individuals commiting fraud on the application level tend to only make minor changes to fields that ussually are distinct such as address and phone number, but to pour over 1,000's of variables to catch one or two individuals is inefficient. So I wnated to build a model to calculate the similarity between two strings and given a threshold would alert me of when two address were oddly similar to another with a specific timeframe (lets say one or two days). 

## The Code

In [ ]:
# Import numpy to perform the matrix algebra necessary to calculate the fuzzy match
import numpy as np
# Define a function that will become the fuzzy match
# I decided to use Levenshtein Distance due to the formulas ability to handle string comparisons of two unique lengths
def string_match(seq1, seq2, ratio_calc = False):
    """ levenshtein_ratio_and_distance:
        Calculates levenshtein distance between two strings.
        If ratio_calc = True, the function computes the
        levenshtein distance ratio of similarity between two strings
        For all i and j, distance[i,j] will contain the Levenshtein
        distance between the first i characters of seq1 and the
        first j characters of seq2
    """
    # Initialize matrix of zeros
    rows = len(seq1)+1
    cols = len(seq2)+1
    distance = np.zeros((rows,cols),dtype = int)

    # Populate matrix of zeros with the indeces of each character of both strings
    for i in range(1, rows):
        for k in range(1,cols):
            distance[i][0] = i
            distance[0][k] = k

    # loop through the matrix to compute the cost of deletions,insertions and/or substitutions    
    for col in range(1, cols):
        for row in range(1, rows):
            if seq1[row-1] == seq2[col-1]:
                cost = 0 # If the characters are the same in the two strings in a given position [i,j] then the cost is 0
            else:
                # In order to align the results with those of the Python Levenshtein package, if we choose to calculate the ratio
                # the cost of a substitution is 2. If we calculate just distance, then the cost of a substitution is 1.
                if ratio_calc == True:
                    cost = 2
                else:
                    cost = 1
            distance[row][col] = min(distance[row-1][col] + 1,      # Cost of deletions
                                 distance[row][col-1] + 1,          # Cost of insertions
                                 distance[row-1][col-1] + cost)     # Cost of substitutions
    if ratio_calc == True:
        # Computation of the Levenshtein Distance Ratio
        Ratio = round(((len(seq1)+len(seq2)) - distance[row][col]) / (len(seq1)+len(seq2)) * 100, 2)
        return "The addresses are {}% similar".format(Ratio)
    else:
        # print(distance) # Uncomment if you want to see the matrix showing how the algorithm computes the cost of deletions,
        # insertions and/or substitutions
        # This is the minimum number of edits needed to convert seq1 to seq2
        return "The strings are {} edits away".format(distance[row][col])

In [2]:
Prev_addrs = ["8847 N Main St",
              "9763 Peachtree blvd",
              "543 State steet"
             ]
target_addr = "10 Main St"
for addr in Prev_addrs:
    distance = string_match(target_addr, addr, ratio_calc = True)
    print (distance)

The addresses are 66.67% similar
The addresses are 20.69% similar
The addresses are 32.0% similar


In [8]:
seq1 = "847 Data Drive"
seq2 = "842 Data Drive"
Distance = string_match(seq1, seq2)
ratio = string_match(seq1, seq2, ratio_calc = True)

In [9]:
print(Distance)
print(ratio)

The strings are 1 edits away
The addresses are 92.86% similar


## Next Steps

Now that I have a functioning model I need to test and get it production ready. So next steps would include:

    1) getting a pandas dataframe of a 100 different address and converting into a list. and comparing the list to a target address. Then testing it on larger data to see how fast and accurate it performs.
    2) Finding away to randomly swap the target address once its finished being compared to the list.
    3) writing more efficient code but given this was just an excerise I wont worry to much about that. 